# Get Insights from Customer Reviews

## Overview

This template demonstrates a complete workflow for analyzing customer feedback using Snowflake Cortex AI functions. You'll learn how to go from raw, unstructured reviews to actionable insights by using SQL to perform sentiment analysis, issue extraction, classification, and even automated response generation.

## Step 1: Set up your environment and data

Let's begin by running the query below. It sets the correct role and warehouse for this session and creates a new schema. It also creates and populates two tables, `PRODUCT_REVIEWS` and `PRODUCT_CATALOG`, with sample data for our analysis.

In [ ]:
USE ROLE SNOWFLAKE_LEARNING_ROLE;
USE DATABASE SNOWFLAKE_LEARNING_DB;
USE WAREHOUSE SNOWFLAKE_LEARNING_WH;

CREATE OR REPLACE SCHEMA CORTEX_REVIEWS_DEMO;
USE SCHEMA CORTEX_REVIEWS_DEMO;

-- Create sample tables
CREATE OR REPLACE TABLE product_catalog (product_id INT, product_name VARCHAR, category VARCHAR);
CREATE OR REPLACE TABLE product_reviews (review_id INT, product_id INT, rating INT, review_text VARCHAR);

-- Insert sample data
INSERT INTO product_catalog (product_id, product_name, category) VALUES
(1, 'E-Verse Smartphone', 'Electronics'),
(2, 'Aura Smartwatch', 'Electronics'),
(3, 'Comfy-Fit T-Shirt', 'Clothing'),
(4, 'Trekker Jeans', 'Clothing');

INSERT INTO product_reviews (review_id, product_id, rating, review_text) VALUES
(101, 1, 1, 'The battery life is terrible, it barely lasts a few hours.'),
(102, 1, 2, 'Camera quality is not as advertised. Very grainy photos.'),
(103, 2, 5, 'Love my new watch, the screen is so crisp and bright!'),
(104, 3, 2, 'I ordered a medium but it feels like a small. The sizing is completely off.'),
(105, 3, 3, 'The color faded after just one wash. Disappointing quality.'),
(106, 4, 5, 'These are the best jeans I have ever owned. Perfect fit!');

SELECT 'Setup is complete. You now have sample review data to analyze.' AS note;

## Step 2: Correlate sentiment with ratings

As a first step, let's perform a quick sanity check. We'll use the `SNOWFLAKE.CORTEX.SENTIMENT` function to score the sentiment of each review. We can then check its correlation with the user-provided star rating to see if they align.

In [ ]:
WITH EXTRACTED_SENTIMENT AS (
    SELECT 
        RATING,
        SNOWFLAKE.CORTEX.SENTIMENT(REVIEW_TEXT) AS SENTIMENT
    FROM PRODUCT_REVIEWS
)
SELECT CORR(SENTIMENT, RATING) AS SENTIMENT_RATING_CORRELATION
FROM EXTRACTED_SENTIMENT;

## Step 3: Find top issues in a category

Now, let's dig deeper. Suppose you want to know what the biggest complaints are for 'Electronics'. You can use `AI_AGG` to analyze all relevant reviews and aggregate the common themes into a single summary.

In [ ]:
SELECT 
  AI_AGG(
    REVIEW_TEXT, 
    'What are the top 3 most common product issues reported in these reviews?'
  ) AS TOP_ISSUES
FROM PRODUCT_REVIEWS pr
JOIN PRODUCT_CATALOG pc ON pr.product_id = pc.product_id
WHERE pc.category = 'Electronics';

## Step 4: Classify reviews by issue type

Once you've identified common issues like 'Sizing' or 'Color,' you can use `AI_CLASSIFY` to build a pipeline that automatically categorizes new reviews. This helps you systematically track and report on known problems.

In [ ]:
WITH CLOTHING_REVIEWS AS (
  SELECT REVIEW_ID, REVIEW_TEXT
  FROM PRODUCT_REVIEWS pr
  JOIN PRODUCT_CATALOG pc ON pr.product_id = pc.product_id
  WHERE pc.category = 'Clothing'
)
SELECT 
    REVIEW_ID,
    REVIEW_TEXT,
    AI_CLASSIFY(
      REVIEW_TEXT, 
      ['Sizing issue', 'Color issue', 'Fabric quality', 'Washing problem']
    ) :labels[0] ::TEXT AS ISSUE_CATEGORY
  FROM CLOTHING_REVIEWS;

## Step 5: Generate responses to customer complaints

Finally, let's close the loop. You can use `AI_COMPLETE` to help your support team draft empathetic and relevant responses to negative reviews, improving customer satisfaction at scale.

In [ ]:
WITH NEGATIVE_CLOTHING_REVIEWS AS (
  SELECT REVIEW_ID, REVIEW_TEXT
  FROM PRODUCT_REVIEWS pr
  JOIN PRODUCT_CATALOG pc ON pr.product_id = pc.product_id
  WHERE pc.category = 'Clothing' AND pr.rating <= 3
)
SELECT 
    REVIEW_TEXT,
    AI_COMPLETE('llama3-8b', 'Draft a concise and apologetic response to the customer complaint below. Only include the draft: ' || REVIEW_TEXT) AS DRAFT_RESPONSE
  FROM NEGATIVE_CLOTHING_REVIEWS;

## Key Takeaways

* **End-to-End Workflow**: You can chain Cortex AI functions together (`SENTIMENT` -> `AI_AGG` -> `AI_CLASSIFY` -> `AI_COMPLETE`) to build a powerful analysis pipeline entirely within Snowflake.
* **Insight from Unstructured Data**: You don't need complex data science tools to extract valuable insights from text. All of this was done with familiar SQL.
* **Automate and Scale**: By identifying common issues and creating classifiers, you can automate the process of tracking feedback and responding to customers more efficiently.

## Additional Resources

* [Documentation: Cortex AI SQL Functions](https://docs.snowflake.com/en/sql-reference/functions-iceberg)
* [More Templates](https://app.snowflake.com/templates)